In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#load dataset 
electricity = pd.read_csv("../input/bdgp2-further-cleaned-datasets/electricity_cleaned_new.csv")

In [ ]:
electricity.dtypes

In [ ]:
#change to DateTime format
electricity["timestamp"] = pd.to_datetime(electricity["timestamp"], format = "%Y-%m-%d %H:%M:%S")

In [ ]:
electricity = electricity.drop("Unnamed: 0", axis=1)

In [ ]:
#set time as index 
electricity = electricity.set_index("timestamp")

In [ ]:
electricity.head()

In [ ]:
#average by week
electricity = electricity.resample("W").mean()

In [ ]:
#electricity
panther_electricity = pd.DataFrame()
P = [col for col in electricity.columns if 'Panther' in col]
panther_electricity[P] = electricity[P]

robin_electricity = pd.DataFrame()
R = [col for col in electricity.columns if 'Robin' in col]
robin_electricity[R] = electricity[R]

fox_electricity = pd.DataFrame()
F = [col for col in electricity.columns if 'Fox' in col]
fox_electricity[F] = electricity[F]

rat_electricity = pd.DataFrame()
R = [col for col in electricity.columns if 'Rat' in col]
rat_electricity[R] = electricity[R]

bear_electricity = pd.DataFrame()
B = [col for col in electricity.columns if 'Bear' in col]
bear_electricity[B] = electricity[B]

lamb_electricity = pd.DataFrame()
L = [col for col in electricity.columns if 'Lamb' in col]
lamb_electricity[L] = electricity[L]

peacock_electricity = pd.DataFrame()
p = [col for col in electricity.columns if 'Peacock' in col]
peacock_electricity[p] = electricity[p]

moose_electricity = pd.DataFrame()
M = [col for col in electricity.columns if 'Moose' in col]
moose_electricity[M] = electricity[M]

gator_electricity = pd.DataFrame()
G = [col for col in electricity.columns if 'Gator' in col]
gator_electricity[G] = electricity[G]

bull_electricity = pd.DataFrame()
B = [col for col in electricity.columns if 'Bull' in col]
bull_electricity[B] = electricity[B]

bobcat_electricity = pd.DataFrame()
b = [col for col in electricity.columns if 'Bobcat' in col]
bobcat_electricity[b] = electricity[b]

crow_electricity = pd.DataFrame()
cr = [col for col in electricity.columns if 'Crow' in col]
crow_electricity[cr] = electricity[cr]

shrew_electricity = pd.DataFrame()
S = [col for col in electricity.columns if 'Shrew' in col]
shrew_electricity[S] = electricity[S]

wolf_electricity = pd.DataFrame()
W = [col for col in electricity.columns if 'Wolf' in col]
wolf_electricity[W] = electricity[W]

hog_electricity = pd.DataFrame()
H = [col for col in electricity.columns if 'Hog' in col]
hog_electricity[H] = electricity[H]

eagle_electricity = pd.DataFrame()
E = [col for col in electricity.columns if 'Eagle' in col]
eagle_electricity[E] = electricity[E]

cockatoo_electricity = pd.DataFrame()
C = [col for col in electricity.columns if 'Cockatoo' in col]
cockatoo_electricity[C] = electricity[C]

mouse_electricity = pd.DataFrame()
M = [col for col in electricity.columns if 'Mouse' in col]
mouse_electricity[M] = electricity[M]

sites_electricity = [panther_electricity, robin_electricity, fox_electricity, rat_electricity, bear_electricity, 
                     lamb_electricity, peacock_electricity, moose_electricity, gator_electricity, bull_electricity,
                     bobcat_electricity, crow_electricity, shrew_electricity, wolf_electricity, hog_electricity,
                     eagle_electricity, cockatoo_electricity, mouse_electricity]

In [ ]:
#Sum up the uage of each location per week for each type of usage and create new column 
for site in sites_electricity:
    name = site.columns[0].split("_")[0]
    site["{}_electricity_sum".format(name)] = site.sum(axis = 1)

# <u> Modelling by Location </u> 

In [ ]:
! pip install pmdarima

In [ ]:
pip install --upgrade pip

In [ ]:
from sklearn import metrics
from scipy.stats import norm
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
import pmdarima as pm

# Panther

In [ ]:
model_data = panther_electricity[["Panther_electricity_sum"]]
train = model_data.iloc[0:(len(model_data)-53)].copy()
test = model_data.iloc[len(train):(len(model_data) -1)].copy()

In [ ]:
sm.graphics.tsa.plot_pacf(train,lags=30)
plt.show()

In [ ]:
sm.graphics.tsa.plot_acf(train,lags=50)
plt.show()

In [ ]:
decomp = seasonal_decompose(panther_electricity["Panther_electricity_sum"], period=12)
decomp.plot()

In [ ]:
result = adfuller(panther_electricity["Panther_electricity_sum"])
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

p-value is not significant while checking for stationarity

* This indicates that statistical properities of data, such as standard deviation, mean and variance is not constant over time

In [ ]:
smodel = pm.auto_arima(train, start_p=1, start_q=1, test="adf", max_p=3, max_q=3, m=12, d = 1, D = 1, start_P=0, seasonal = True, trace=True,
                      error_action="ignore", suppress_warnings=True, stepwise=True
                      )

smodel.summary()

From the auto arima function, the best model is SARIMAX(0, 1, 1) x (0, 1, [1,2], 12)

In [ ]:
mod = sm.tsa.statespace.SARIMAX(train, order=(0,1,1), seasonal_order=(0,1,[1,2],12))
mod_fit = mod.fit()
mod_fit.summary()

In [ ]:
pred = mod_fit.predict()
test["pred"] = pred.values
test.head()

In [ ]:
test["Panther_electricity_sum"].plot(figsize=(25,10),color = 'red')
test["pred"].plot()
plt.show()

In [ ]:
test['residual'] = abs(test['Panther_electricity_sum']-test['pred'])
MAE = test['residual'].sum()/len(test)
MAPE = (abs(test['residual'])/test['Panther_electricity_sum']).sum()*100/len(test)
print("MAE:", MAE)
print("MAPE:", MAPE)

Not an ideal prediction model

# Robin

In [ ]:
model_data = robin_electricity[["Robin_electricity_sum"]]
train = model_data.iloc[0:(len(model_data)-53)].copy()
test = model_data.iloc[len(train):(len(model_data) -1)].copy()

In [ ]:
sm.graphics.tsa.plot_pacf(train,lags=30)
plt.show()

In [ ]:
sm.graphics.tsa.plot_acf(train,lags=30)
plt.show()

In [ ]:
decomp = seasonal_decompose(robin_electricity["Robin_electricity_sum"], period=12)
decomp.plot()

In [ ]:
result = adfuller(robin_electricity["Robin_electricity_sum"])
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

p-value is significant while checking for stationarity

* This indicates that statistical properities of data, such as standard deviation, mean and variance is constant over time

In [ ]:
smodel = pm.auto_arima(train, start_p=1, start_q=1, test="adf", max_p=3, max_q=3, m=12, d = 1, D = 1, start_P=0, seasonal = True, trace=True,
                      error_action="ignore", suppress_warnings=True, stepwise=True
                      )

smodel.summary()

From the auto arima function, the best model is SARIMAX(0, 1, 2) x (0, 1, [1], 12)

In [ ]:
mod = sm.tsa.statespace.SARIMAX(train, order=(0,1,2), seasonal_order=(0,1,[1],12))
mod_fit = mod.fit()
mod_fit.summary()

In [ ]:
pred = mod_fit.predict()
test["pred"] = pred.values
test.head()

In [ ]:
test["Robin_electricity_sum"].plot(figsize=(25,10),color = 'red')
test["pred"].plot()
plt.show()

In [ ]:
test['residual'] = abs(test['Robin_electricity_sum']-test['pred'])
MAE = test['residual'].sum()/len(test)
MAPE = (abs(test['residual'])/test['Robin_electricity_sum']).sum()*100/len(test)
print("MAE:", MAE)
print("MAPE:", MAPE)

# Fox

In [ ]:
model_data = fox_electricity[["Fox_electricity_sum"]]
train = model_data.iloc[0:(len(model_data)-53)].copy()
test = model_data.iloc[len(train):(len(model_data) -1)].copy()

In [ ]:
sm.graphics.tsa.plot_pacf(train,lags=30)
plt.show()

In [ ]:
sm.graphics.tsa.plot_acf(train,lags=30)
plt.show()

In [ ]:
decomp = seasonal_decompose(fox_electricity["Fox_electricity_sum"], period=12)
decomp.plot()

In [ ]:
result = adfuller(fox_electricity["Fox_electricity_sum"])
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

p-value is significant while checking for stationarity

* This indicates that statistical properities of data, such as standard deviation, mean and variance is constant over time

In [ ]:
smodel = pm.auto_arima(train, start_p=1, start_q=1, test="adf", max_p=3, max_q=3, m=12, d = 1, D = 1, start_P=0, seasonal = True, trace=True,
                      error_action="ignore", suppress_warnings=True, stepwise=True
                      )

smodel.summary()

From the auto arima function, the best model is SARIMAX(0, 1, 0) x (1, 1, [1], 12)

In [ ]:
mod = sm.tsa.statespace.SARIMAX(train, order=(0,1,0), seasonal_order=(1,1,[1],12))
mod_fit = mod.fit()
mod_fit.summary()

In [ ]:
pred = mod_fit.predict()
test["pred"] = pred.values
test.head()

In [ ]:
test["Fox_electricity_sum"].plot(figsize=(25,10),color = 'red')
test["pred"].plot()
plt.show()

In [ ]:
test['residual'] = abs(test['Fox_electricity_sum']-test['pred'])
MAE = test['residual'].sum()/len(test)
MAPE = (abs(test['residual'])/test['Fox_electricity_sum']).sum()*100/len(test)
print("MAE:", MAE)
print("MAPE:", MAPE)

# Rat

In [ ]:
model_data = rat_electricity[["Rat_electricity_sum"]]
train = model_data.iloc[0:(len(model_data)-53)].copy()
test = model_data.iloc[len(train):(len(model_data) -1)].copy()

In [ ]:
sm.graphics.tsa.plot_pacf(train,lags=30)
plt.show()

In [ ]:
sm.graphics.tsa.plot_acf(train,lags=30)
plt.show()

In [ ]:
decomp = seasonal_decompose(rat_electricity["Rat_electricity_sum"], period=12)
decomp.plot()

In [ ]:
result = adfuller(rat_electricity["Rat_electricity_sum"])
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

p-value is not significant while checking for stationarity

* This indicates that statistical properities of data, such as standard deviation, mean and variance is not constant over time

In [ ]:
smodel = pm.auto_arima(train, start_p=1, start_q=1, test="adf", max_p=3, max_q=3, m=12, d = 1, D = 1, start_P=0, seasonal = True, trace=True,
                      error_action="ignore", suppress_warnings=True, stepwise=True
                      )

smodel.summary()

From the auto arima function, the best model is SARIMAX(0, 1, 0) x (0, 1, [1], 12)

In [ ]:
mod = sm.tsa.statespace.SARIMAX(train, order=(0,1,0), seasonal_order=(0,1,[1],12))
mod_fit = mod.fit()
mod_fit.summary()

In [ ]:
pred = mod_fit.predict()
test["pred"] = pred.values
test.head()

In [ ]:
test["Rat_electricity_sum"].plot(figsize=(25,10),color = 'red')
test["pred"].plot()
plt.show()

In [ ]:
test['residual'] = abs(test['Rat_electricity_sum']-test['pred'])
MAE = test['residual'].sum()/len(test)
MAPE = (abs(test['residual'])/test['Rat_electricity_sum']).sum()*100/len(test)
print("MAE:", MAE)
print("MAPE:", MAPE)

MAE is not ideal. 

# Bear

In [ ]:
model_data = bear_electricity[["Bear_electricity_sum"]]
train = model_data.iloc[0:(len(model_data)-53)].copy()
test = model_data.iloc[len(train):(len(model_data) -1)].copy()

In [ ]:
sm.graphics.tsa.plot_pacf(train,lags=30)
plt.show()

In [ ]:
sm.graphics.tsa.plot_acf(train,lags=30)
plt.show()

In [ ]:
decomp = seasonal_decompose(bear_electricity["Bear_electricity_sum"], period=12)
decomp.plot()

In [ ]:
result = adfuller(bear_electricity["Bear_electricity_sum"])
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

p-value is significant while checking for stationarity

* This indicates that statistical properities of data, such as standard deviation, mean and variance is constant over time

In [ ]:
smodel = pm.auto_arima(train, start_p=1, start_q=1, test="adf", max_p=3, max_q=3, m=12, d = 1, D = 1, start_P=0, seasonal = True, trace=True,
                      error_action="ignore", suppress_warnings=True, stepwise=True
                      )

smodel.summary()

From the auto arima function, the best model is SARIMAX(0, 1, 0) x (0, 1, [1], 12)

In [ ]:
mod = sm.tsa.statespace.SARIMAX(train, order=(0,1,0), seasonal_order=(0,1,[1],12))
mod_fit = mod.fit()
mod_fit.summary()

In [ ]:
pred = mod_fit.predict()
test["pred"] = pred.values
test.head()

In [ ]:
test["Bear_electricity_sum"].plot(figsize=(25,10),color = 'red')
test["pred"].plot()
plt.show()

In [ ]:
test['residual'] = abs(test['Bear_electricity_sum']-test['pred'])
MAE = test['residual'].sum()/len(test)
MAPE = (abs(test['residual'])/test['Bear_electricity_sum']).sum()*100/len(test)
print("MAE:", MAE)
print("MAPE:", MAPE)

# Lamb

In [ ]:
model_data = lamb_electricity[["Lamb_electricity_sum"]]
train = model_data.iloc[0:(len(model_data)-53)].copy()
test = model_data.iloc[len(train):(len(model_data) -1)].copy()

In [ ]:
sm.graphics.tsa.plot_pacf(train,lags=20)
plt.show()

In [ ]:
sm.graphics.tsa.plot_acf(train,lags=30)
plt.show()

In [ ]:
decomp = seasonal_decompose(bear_electricity["Bear_electricity_sum"], period=12)
decomp.plot()

In [ ]:
result = adfuller(lamb_electricity["Lamb_electricity_sum"])
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

p-value is not significant while checking for stationarity

* This indicates that statistical properities of data, such as standard deviation, mean and variance is not constant over time

In [ ]:
smodel = pm.auto_arima(train, start_p=1, start_q=1, test="adf", max_p=3, max_q=3, m=12, d = 1, D = 1, start_P=0, seasonal = True, trace=True,
                      error_action="ignore", suppress_warnings=True, stepwise=True
                      )

smodel.summary()

From the auto arima function, the best model is SARIMAX(0, 1, 0) x (0, 1, [1], 12)

In [ ]:
mod = sm.tsa.statespace.SARIMAX(train, order=(0,1,0), seasonal_order=(0,1,[1],12))
mod_fit = mod.fit()
mod_fit.summary()

In [ ]:
pred = mod_fit.predict()
test["pred"] = pred.values
test.head()

In [ ]:
test["Lamb_electricity_sum"].plot(figsize=(25,10),color = 'red')
test["pred"].plot()
plt.show()

In [ ]:
test['residual'] = abs(test['Lamb_electricity_sum']-test['pred'])
MAE = test['residual'].sum()/len(test)
MAPE = (abs(test['residual'])/test['Lamb_electricity_sum']).sum()*100/len(test)
print("MAE:", MAE)
print("MAPE:", MAPE)

Metrics not ideal. Model needs adjusting. 

# Peacock

In [ ]:
model_data = peacock_electricity[["Peacock_electricity_sum"]]
train = model_data.iloc[0:(len(model_data)-53)].copy()
test = model_data.iloc[len(train):(len(model_data) -1)].copy()

In [ ]:
sm.graphics.tsa.plot_pacf(train,lags=30)
plt.show()

In [ ]:
sm.graphics.tsa.plot_acf(train,lags=30)
plt.show()

In [ ]:
decomp = seasonal_decompose(peacock_electricity["Peacock_electricity_sum"], period=12)
decomp.plot()

In [ ]:
result = adfuller(peacock_electricity["Peacock_electricity_sum"])
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

p-value is significant while checking for stationarity

* This indicates that statistical properities of data, such as standard deviation, mean and variance is constant over time

In [ ]:
smodel = pm.auto_arima(train, start_p=1, start_q=1, test="adf", max_p=3, max_q=3, m=12, d = 1, D = 1, start_P=0, seasonal = True, trace=True,
                      error_action="ignore", suppress_warnings=True, stepwise=True
                      )

smodel.summary()

From the auto arima function, the best model is SARIMAX(0, 1, 0) x (0, 1, 0, 12)

In [ ]:
mod = sm.tsa.statespace.SARIMAX(train, order=(0,1,0), seasonal_order=(0,1,0,12))
mod_fit = mod.fit()
mod_fit.summary()

In [ ]:
pred = mod_fit.predict()
test["pred"] = pred.values
test.head()

In [ ]:
test["Peacock_electricity_sum"].plot(figsize=(25,10),color = 'red')
test["pred"].plot()
plt.show()

In [ ]:
test['residual'] = abs(test['Peacock_electricity_sum']-test['pred'])
MAE = test['residual'].sum()/len(test)
MAPE = (abs(test['residual'])/test['Peacock_electricity_sum']).sum()*100/len(test)
print("MAE:", MAE)
print("MAPE:", MAPE)

# Moose 

In [ ]:
model_data = moose_electricity[["Moose_electricity_sum"]]
train = model_data.iloc[0:(len(model_data)-53)].copy()
test = model_data.iloc[len(train):(len(model_data) -1)].copy()

In [ ]:
sm.graphics.tsa.plot_pacf(train,lags=30)
plt.show()

In [ ]:
sm.graphics.tsa.plot_acf(train,lags=30)
plt.show()

In [ ]:
decomp = seasonal_decompose(moose_electricity["Moose_electricity_sum"], period=12)
decomp.plot()

In [ ]:
result = adfuller(moose_electricity["Moose_electricity_sum"])
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

p-value is not significant while checking for stationarity

* This indicates that statistical properities of data, such as standard deviation, mean and variance is not constant over time

In [ ]:
smodel = pm.auto_arima(train, start_p=1, start_q=1, test="adf", max_p=3, max_q=3, m=12, d = 1, D = 1, start_P=0, seasonal = True, trace=True,
                      error_action="ignore", suppress_warnings=True, stepwise=True
                      )

smodel.summary()

From the auto arima function, the best model is SARIMAX(0, 1, 0) x (0, 1, [1], 12)

In [ ]:
mod = sm.tsa.statespace.SARIMAX(train, order=(0,1,0), seasonal_order=(0,1,[1],12))
mod_fit = mod.fit()
mod_fit.summary()

In [ ]:
pred = mod_fit.predict()
test["pred"] = pred.values
test.head()

In [ ]:
test["Moose_electricity_sum"].plot(figsize=(25,10),color = 'red')
test["pred"].plot()
plt.show()

In [ ]:
test['residual'] = abs(test['Moose_electricity_sum']-test['pred'])
MAE = test['residual'].sum()/len(test)
MAPE = (abs(test['residual'])/test['Moose_electricity_sum']).sum()*100/len(test)
print("MAE:", MAE)
print("MAPE:", MAPE)

# Gator

In [ ]:
model_data = gator_electricity[["Gator_electricity_sum"]]
train = model_data.iloc[0:(len(model_data)-53)].copy()
test = model_data.iloc[len(train):(len(model_data) -1)].copy()

In [ ]:
sm.graphics.tsa.plot_pacf(train,lags=30)
plt.show()

In [ ]:
sm.graphics.tsa.plot_acf(train,lags=30)
plt.show()

In [ ]:
decomp = seasonal_decompose(gator_electricity["Gator_electricity_sum"], period=12)
decomp.plot()

In [ ]:
result = adfuller(gator_electricity["Gator_electricity_sum"])
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

p-value is significant while checking for stationarity

* This indicates that statistical properities of data, such as standard deviation, mean and variance is constant over time

In [ ]:
smodel = pm.auto_arima(train, start_p=1, start_q=1, test="adf", max_p=3, max_q=3, m=12, d = 1, D = 1, start_P=0, seasonal = True, trace=True,
                      error_action="ignore", suppress_warnings=True, stepwise=True
                      )

smodel.summary()

From the auto arima function, the best model is SARIMAX(1, 1, 1) x (0, 1, 1, 12)

In [ ]:
mod = sm.tsa.statespace.SARIMAX(train, order=(1,1,1), seasonal_order=(0,1,1,12))
mod_fit = mod.fit()
mod_fit.summary()

In [ ]:
pred = mod_fit.predict()
test["pred"] = pred.values
test.head()

In [ ]:
test["Gator_electricity_sum"].plot(figsize=(25,10),color = 'red')
test["pred"].plot()
plt.show()

In [ ]:
test['residual'] = abs(test['Gator_electricity_sum']-test['pred'])
MAE = test['residual'].sum()/len(test)
MAPE = (abs(test['residual'])/test['Gator_electricity_sum']).sum()*100/len(test)
print("MAE:", MAE)
print("MAPE:", MAPE)

# Bull

In [ ]:
model_data = bull_electricity[["Bull_electricity_sum"]]
train = model_data.iloc[0:(len(model_data)-53)].copy()
test = model_data.iloc[len(train):(len(model_data) -1)].copy()

In [ ]:
sm.graphics.tsa.plot_pacf(train,lags=30)
plt.show()

In [ ]:
sm.graphics.tsa.plot_acf(train,lags=30)
plt.show()

In [ ]:
decomp = seasonal_decompose(bull_electricity["Bull_electricity_sum"], period=12)
decomp.plot()

In [ ]:
result = adfuller(bull_electricity["Bull_electricity_sum"])
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

p-value is significant while checking for stationarity

* This indicates that statistical properities of data, such as standard deviation, mean and variance is constant over time

In [ ]:
smodel = pm.auto_arima(train, start_p=1, start_q=1, test="adf", max_p=3, max_q=3, m=12, d = 1, D = 1, start_P=0, seasonal = True, trace=True,
                      error_action="ignore", suppress_warnings=True, stepwise=True
                      )

smodel.summary()

From the auto arima function, the best model is SARIMAX(1, 1, 1) x (0, 1, 1, 12)

In [ ]:
mod = sm.tsa.statespace.SARIMAX(train, order=(1,1,1), seasonal_order=(0,1,1,12))
mod_fit = mod.fit()
mod_fit.summary()

In [ ]:
pred = mod_fit.predict()
test["pred"] = pred.values
test.head()

In [ ]:
test["Bull_electricity_sum"].plot(figsize=(25,10),color = 'red')
test["pred"].plot()
plt.show()

In [ ]:
test['residual'] = abs(test['Bull_electricity_sum']-test['pred'])
MAE = test['residual'].sum()/len(test)
MAPE = (abs(test['residual'])/test['Bull_electricity_sum']).sum()*100/len(test)
print("MAE:", MAE)
print("MAPE:", MAPE)

Metrics are not ideal.

# Bobcat

In [ ]:
model_data = bobcat_electricity[["Bobcat_electricity_sum"]]
train = model_data.iloc[0:(len(model_data)-53)].copy()
test = model_data.iloc[len(train):(len(model_data) -1)].copy()

In [ ]:
sm.graphics.tsa.plot_pacf(train,lags=30)
plt.show()

In [ ]:
sm.graphics.tsa.plot_acf(train,lags=30)
plt.show()

In [ ]:
decomp = seasonal_decompose(bobcat_electricity["Bobcat_electricity_sum"], period=12)
decomp.plot()

In [ ]:
result = adfuller(bobcat_electricity["Bobcat_electricity_sum"])
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

p-value is significant while checking for stationarity

* This indicates that statistical properities of data, such as standard deviation, mean and variance is constant over time

In [ ]:
smodel = pm.auto_arima(train, start_p=1, start_q=1, test="adf", max_p=3, max_q=3, m=12, d = 1, D = 1, start_P=0, seasonal = True, trace=True,
                      error_action="ignore", suppress_warnings=True, stepwise=True
                      )

smodel.summary()

From the auto arima function, the best model is SARIMAX(2, 1, 0) x (0, 1, [1], 12)

In [ ]:
mod = sm.tsa.statespace.SARIMAX(train, order=(2,1,0), seasonal_order=(0,1,[1],12))
mod_fit = mod.fit()
mod_fit.summary()

In [ ]:
pred = mod_fit.predict()
test["pred"] = pred.values
test.head()


In [ ]:
test["Bobcat_electricity_sum"].plot(figsize=(25,10),color = 'red')
test["pred"].plot()
plt.show()

In [ ]:
test['residual'] = abs(test['Bobcat_electricity_sum']-test['pred'])
MAE = test['residual'].sum()/len(test)
MAPE = (abs(test['residual'])/test['Bobcat_electricity_sum']).sum()*100/len(test)
print("MAE:", MAE)
print("MAPE:", MAPE)

# Crow 

In [ ]:
model_data = crow_electricity[["Crow_electricity_sum"]]
train = model_data.iloc[0:(len(model_data)-53)].copy()
test = model_data.iloc[len(train):(len(model_data) -1)].copy()

In [ ]:
sm.graphics.tsa.plot_pacf(train,lags=20)
plt.show()

In [ ]:
sm.graphics.tsa.plot_acf(train,lags=30)
plt.show()

In [ ]:
decomp = seasonal_decompose(crow_electricity["Crow_electricity_sum"], period=12)
decomp.plot()

In [ ]:
result = adfuller(crow_electricity["Crow_electricity_sum"])
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

p-value is not significant while checking for stationarity

* This indicates that statistical properities of data, such as standard deviation, mean and variance is not constant over time

In [ ]:
smodel = pm.auto_arima(train, start_p=1, start_q=1, test="adf", max_p=3, max_q=3, m=12, d = 1, D = 1, start_P=0, seasonal = True, trace=True,
                      error_action="ignore", suppress_warnings=True, stepwise=True
                      )

smodel.summary()

From the auto arima function, the best model is SARIMAX(0, 1, 0) x (2, 1, 0, 12)

In [ ]:
mod = sm.tsa.statespace.SARIMAX(train, order=(0,1,0), seasonal_order=(2,1,0,12))
mod_fit = mod.fit()
mod_fit.summary()

In [ ]:
pred = mod_fit.predict()
test["pred"] = pred.values
test.head()

In [ ]:
test["Crow_electricity_sum"].plot(figsize=(25,10),color = 'red')
test["pred"].plot()
plt.show()

In [ ]:
test['residual'] = abs(test['Crow_electricity_sum']-test['pred'])
MAE = test['residual'].sum()/len(test)
MAPE = (abs(test['residual'])/test['Crow_electricity_sum']).sum()*100/len(test)
print("MAE:", MAE)
print("MAPE:", MAPE)

# Shrew

In [ ]:
model_data = shrew_electricity[["Shrew_electricity_sum"]]
train = model_data.iloc[0:(len(model_data)-53)].copy()
test = model_data.iloc[len(train):(len(model_data) -1)].copy()

In [ ]:
sm.graphics.tsa.plot_pacf(train,lags=30)
plt.show()

In [ ]:
sm.graphics.tsa.plot_acf(train,lags=30)
plt.show()

In [ ]:
decomp = seasonal_decompose(shrew_electricity["Shrew_electricity_sum"], period=12)
decomp.plot()

In [ ]:
result = adfuller(shrew_electricity["Shrew_electricity_sum"])
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

p-value is not significant while checking for stationarity

* This indicates that statistical properities of data, such as standard deviation, mean and variance is not constant over time

In [ ]:
smodel = pm.auto_arima(train, start_p=1, start_q=1, test="adf", max_p=3, max_q=3, m=12, d = 1, D = 1, start_P=0, seasonal = True, trace=True,
                      error_action="ignore", suppress_warnings=True, stepwise=True
                      )

smodel.summary()

From the auto arima function, the best model is SARIMAX(1, 1, 0) x (0, 1, [1], 12)

In [ ]:
mod = sm.tsa.statespace.SARIMAX(train, order=(1,1,0), seasonal_order=(0,1,[1],12))
mod_fit = mod.fit()
mod_fit.summary()

In [ ]:
pred = mod_fit.predict()
test["pred"] = pred.values
test.head()

In [ ]:
test["Shrew_electricity_sum"].plot(figsize=(25,10),color = 'red')
test["pred"].plot()
plt.show()

In [ ]:
test['residual'] = abs(test['Shrew_electricity_sum']-test['pred'])
MAE = test['residual'].sum()/len(test)
MAPE = (abs(test['residual'])/test['Shrew_electricity_sum']).sum()*100/len(test)
print("MAE:", MAE)
print("MAPE:", MAPE)

Not ideal model. 

# Wolf

In [ ]:
model_data = wolf_electricity[["Wolf_electricity_sum"]]
train = model_data.iloc[0:(len(model_data)-53)].copy()
test = model_data.iloc[len(train):(len(model_data) -1)].copy()

In [ ]:
sm.graphics.tsa.plot_pacf(train,lags=20)
plt.show()

In [ ]:
sm.graphics.tsa.plot_acf(train,lags=30)
plt.show()

In [ ]:
decomp = seasonal_decompose(wolf_electricity["Wolf_electricity_sum"], period=12)
decomp.plot()

In [ ]:
result = adfuller(wolf_electricity["Wolf_electricity_sum"])
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

p-value is significant while checking for stationarity

* This indicates that statistical properities of data, such as standard deviation, mean and variance is constant over time

In [ ]:
smodel = pm.auto_arima(train, start_p=1, start_q=1, test="adf", max_p=3, max_q=3, m=12, d = 1, D = 1, start_P=0, seasonal = True, trace=True,
                      error_action="ignore", suppress_warnings=True, stepwise=True
                      )

smodel.summary()

From the auto arima function, the best model is SARIMAX(3, 1, 0) x (1, 1, 0, 12)

In [ ]:
mod = sm.tsa.statespace.SARIMAX(train, order=(3,1,0), seasonal_order=(1,1,0,12))
mod_fit = mod.fit()
mod_fit.summary()

In [ ]:
pred = mod_fit.predict()
test["pred"] = pred.values
test.head()

In [ ]:
test["Wolf_electricity_sum"].plot(figsize=(25,10),color = 'red')
test["pred"].plot()
plt.show()

In [ ]:
test['residual'] = abs(test['Wolf_electricity_sum']-test['pred'])
MAE = test['residual'].sum()/len(test)
MAPE = (abs(test['residual'])/test['Wolf_electricity_sum']).sum()*100/len(test)
print("MAE:", MAE)
print("MAPE:", MAPE)

# Hog

In [ ]:
model_data = hog_electricity[["Hog_electricity_sum"]]
train = model_data.iloc[0:(len(model_data)-53)].copy()
test = model_data.iloc[len(train):(len(model_data) -1)].copy()

In [ ]:
sm.graphics.tsa.plot_pacf(train,lags=20)
plt.show()

In [ ]:
sm.graphics.tsa.plot_acf(train,lags=30)
plt.show()


In [ ]:
decomp = seasonal_decompose(hog_electricity["Hog_electricity_sum"], period=12)
decomp.plot()

In [ ]:
result = adfuller(hog_electricity["Hog_electricity_sum"])
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

p-value is significant while checking for stationarity

* This indicates that statistical properities of data, such as standard deviation, mean and variance is constant over time

In [ ]:
smodel = pm.auto_arima(train, start_p=1, start_q=1, test="adf", max_p=3, max_q=3, m=12, d = 1, D = 1, start_P=0, seasonal = True, trace=True,
                      error_action="ignore", suppress_warnings=True, stepwise=True
                      )

smodel.summary()

From the auto arima function, the best model is SARIMAX(0, 1, 0) x (0, 1, [1], 12)

In [ ]:
mod = sm.tsa.statespace.SARIMAX(train, order=(0,1,0), seasonal_order=(0,1,[1],12))
mod_fit = mod.fit()
mod_fit.summary()

In [ ]:
pred = mod_fit.predict()
test["pred"] = pred.values
test.head()

In [ ]:
test["Hog_electricity_sum"].plot(figsize=(25,10),color = 'red')
test["pred"].plot()
plt.show()

In [ ]:
test['residual'] = abs(test['Hog_electricity_sum']-test['pred'])
MAE = test['residual'].sum()/len(test)
MAPE = (abs(test['residual'])/test['Hog_electricity_sum']).sum()*100/len(test)
print("MAE:", MAE)
print("MAPE:", MAPE)

# Eagle

In [ ]:
model_data = eagle_electricity[["Eagle_electricity_sum"]]
train = model_data.iloc[0:(len(model_data)-53)].copy()
test = model_data.iloc[len(train):(len(model_data) -1)].copy()

In [ ]:
sm.graphics.tsa.plot_pacf(train,lags=20)
plt.show()

In [ ]:
sm.graphics.tsa.plot_acf(train,lags=30)
plt.show()

In [ ]:
decomp = seasonal_decompose(eagle_electricity["Eagle_electricity_sum"], period=12)
decomp.plot()

In [ ]:
result = adfuller(eagle_electricity["Eagle_electricity_sum"])
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

p-value is significant while checking for stationarity

* This indicates that statistical properities of data, such as standard deviation, mean and variance is constant over time

In [ ]:
smodel = pm.auto_arima(train, start_p=1, start_q=1, test="adf", max_p=3, max_q=3, m=12, d = 1, D = 1, start_P=0, seasonal = True, trace=True,
                      error_action="ignore", suppress_warnings=True, stepwise=True
                      )

smodel.summary()

From the auto arima function, the best model is SARIMAX(0, 1, 1) x (1, 1, [], 12)

In [ ]:
mod = sm.tsa.statespace.SARIMAX(train, order=(0,1,1), seasonal_order=(1,1,[],12))
mod_fit = mod.fit()
mod_fit.summary()

In [ ]:
pred = mod_fit.predict()
test["pred"] = pred.values
test.head()

In [ ]:
test["Eagle_electricity_sum"].plot(figsize=(25,10),color = 'red')
test["pred"].plot()
plt.show()

In [ ]:
test['residual'] = abs(test['Eagle_electricity_sum']-test['pred'])
MAE = test['residual'].sum()/len(test)
MAPE = (abs(test['residual'])/test['Eagle_electricity_sum']).sum()*100/len(test)
print("MAE:", MAE)
print("MAPE:", MAPE)

# Cockatoo

In [ ]:
model_data = cockatoo_electricity[["Cockatoo_electricity_sum"]]
train = model_data.iloc[0:(len(model_data)-53)].copy()
test = model_data.iloc[len(train):(len(model_data) -1)].copy()

In [ ]:
sm.graphics.tsa.plot_pacf(train,lags=30)
plt.show()

In [ ]:
sm.graphics.tsa.plot_acf(train,lags=30)
plt.show()

In [ ]:
decomp = seasonal_decompose(cockatoo_electricity["Cockatoo_electricity_sum"], period=12)
decomp.plot()

In [ ]:
result = adfuller(cockatoo_electricity["Cockatoo_electricity_sum"])
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

p-value is significant while checking for stationarity

* This indicates that statistical properities of data, such as standard deviation, mean and variance is constant over time

In [ ]:
smodel = pm.auto_arima(train, start_p=1, start_q=1, test="adf", max_p=3, max_q=3, m=12, d = 1, D = 1, start_P=0, seasonal = True, trace=True,
                      error_action="ignore", suppress_warnings=True, stepwise=True
                      )

smodel.summary()

From the auto arima function, the best model is SARIMAX(0, 1, 0) x (0, 1, [1,2], 12)

In [ ]:
mod = sm.tsa.statespace.SARIMAX(train, order=(0,1,0), seasonal_order=(0,1,[1,2],12))
mod_fit = mod.fit()
mod_fit.summary()

In [ ]:
pred = mod_fit.predict()
test["pred"] = pred.values
test.head()

In [ ]:
test["Cockatoo_electricity_sum"].plot(figsize=(25,10),color = 'red')
test["pred"].plot()
plt.show()

In [ ]:
test['residual'] = abs(test['Cockatoo_electricity_sum']-test['pred'])
MAE = test['residual'].sum()/len(test)
MAPE = (abs(test['residual'])/test['Cockatoo_electricity_sum']).sum()*100/len(test)
print("MAE:", MAE)
print("MAPE:", MAPE)

# Mouse

In [ ]:
model_data = mouse_electricity[["Mouse_electricity_sum"]]
train = model_data.iloc[0:(len(model_data)-53)].copy()
test = model_data.iloc[len(train):(len(model_data) -1)].copy()

In [ ]:
sm.graphics.tsa.plot_pacf(train,lags=15)
plt.show()

In [ ]:
sm.graphics.tsa.plot_acf(train,lags=30)
plt.show()

In [ ]:
decomp = seasonal_decompose(mouse_electricity["Mouse_electricity_sum"], period=12)
decomp.plot()

In [ ]:
result = adfuller(mouse_electricity["Mouse_electricity_sum"])
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

p-value is not significant while checking for stationarity

* This indicates that statistical properities of data, such as standard deviation, mean and variance is not constant over time

In [ ]:
smodel = pm.auto_arima(train, start_p=1, start_q=1, test="adf", max_p=3, max_q=3, m=12, d = 1, D = 1, start_P=0, seasonal = True, trace=True,
                      error_action="ignore", suppress_warnings=True, stepwise=True
                      )

smodel.summary()

From the auto arima function, the best model is SARIMAX(0, 1, 1) x (0, 1, 1, 12)

In [ ]:
mod = sm.tsa.statespace.SARIMAX(train, order=(0,1,1), seasonal_order=(0,1,1,12))
mod_fit = mod.fit()
mod_fit.summary()

In [ ]:
pred = mod_fit.predict()
test["pred"] = pred.values
test.head()

In [ ]:
test["Mouse_electricity_sum"].plot(figsize=(25,10),color = 'red')
test["pred"].plot()
plt.show()

In [ ]:
test['residual'] = abs(test['Mouse_electricity_sum']-test['pred'])
MAE = test['residual'].sum()/len(test)
MAPE = (abs(test['residual'])/test['Mouse_electricity_sum']).sum()*100/len(test)
print("MAE:", MAE)
print("MAPE:", MAPE)